In [20]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import gym
from IPython.display import Image
import gym 
import itertools 
import matplotlib 
import matplotlib.style 
import numpy as np 
import pandas as pd 
import sys 
from collections import defaultdict 
import plotting 

In [3]:
env = gym.make('LunarLander-v2')

In [4]:
env.action_space.n

4

In [5]:
env.observation_space.high

array([inf, inf, inf, inf, inf, inf, inf, inf], dtype=float32)

In [6]:
env.observation_space.low

array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf], dtype=float32)

In [7]:
print(env.reset())

[ 0.00480614  1.408443    0.4867928  -0.11010541 -0.00556231 -0.11026593
  0.          0.        ]


In [8]:
print(env.step(1))

(array([ 0.00953903,  1.4053781 ,  0.47693357, -0.13623327, -0.00916664,
       -0.07209262,  0.        ,  0.        ], dtype=float32), 0.22167695872278273, False, {})


In [33]:
def qLearning(env, num_episodes, discount_factor = 1.0, 
                            alpha = 0.6, epsilon = 0.1): 
    """ 
    Q-Learning algorithm: Off-policy TD control. 
    Finds the optimal greedy policy while improving 
    following an epsilon-greedy policy"""
       
    # Action value function 
    # A nested dictionary that maps 
    # state -> (action -> action-value). 
    Q = defaultdict(lambda: np.zeros(env.action_space.n)) 
   
    # Keeps track of useful statistics 
    stats = {}
    stats["episode_lengths"] = np.zeros(num_episodes)
    stats["episode_rewards"] = np.zeros(num_episodes)    
       
    # Create an epsilon greedy policy function 
    # appropriately for environment action space 
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n) 
       
    # For every episode 
    for ith_episode in range(num_episodes): 
           
        # Reset the environment and pick the first action 
        state = env.reset() 
           
        for t in itertools.count(): 
               
            # get probabilities of all actions from current state 
            action_probabilities = policy(state) 
   
            # choose action according to  
            # the probability distribution 
            action = np.random.choice(np.arange( 
                      len(action_probabilities)), 
                       p = action_probabilities) 
   
            # take action and get reward, transit to next state 
            next_state, reward, done, _ = env.step(action) 
   
            # Update statistics 
            stats["episode_rewards"][ith_episode] += reward 
            stats["episode_lengths"][ith_episode] = t 
               
            # TD Update 
            best_next_action = np.argmax(Q[next_state])     
            td_target = reward + discount_factor * Q[next_state][best_next_action] 
            td_delta = td_target - Q[state][action] 
            Q[state][action] += alpha * td_delta 
   
            # done is True if episode terminated    
            if done: 
                break
                   
            state = next_state 
       
    return Q, stats

In [38]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions): 
    """ 
    Creates an epsilon-greedy policy based 
    on a given Q-function and epsilon. 
       
    Returns a function that takes the state 
    as an input and returns the probabilities 
    for each action in the form of a numpy array  
    of length of the action space(set of possible actions). 
    """
    def policyFunction(state): 
   
        Action_probabilities = np.ones(num_actions, 
                dtype = float) * epsilon / num_actions 
                  
        best_action = np.argmax(Q[state]) 
        Action_probabilities[best_action] += (1.0 - epsilon) 
        return Action_probabilities 
   
    return policyFunction 

In [39]:
Q, stats = qLearning(env, 1000) 

TypeError: unhashable type: 'numpy.ndarray'